In [1]:
#Instalacion de dependencias

!pip install "snowflake-connector-python[pandas]"
!pip install "snowflake-connector-python[secure-local-storage,pandas]"
!pip install --upgrade snowflake-sqlalchemy

In [3]:
import warnings
warnings.filterwarnings("ignore")

from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

engine = create_engine(URL(
    account = 'pypoxiq-om82528',
    user = 'BOOTCAMPDA910',
    password = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
    database = 'ecommerce',
    schema = 'RAW_FILES',
    warehouse = 'COMPUTE_WH',
    role='ACCOUNTADMIN',
    #session_parameters={}
))

try:
    connection = engine.connect()
    results = connection.execute('select current_version()').fetchone()
    print('Conectado a Snowflake v{}!'.format(results[0]) )
finally:
    connection.close()
    engine.dispose()

Conectado a Snowflake v7.34.0!


In [4]:
from snowflake.sqlalchemy import URL
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
import pandas as pd
 

def fix_date_cols(df, tz='UTC'):
    cols = df.select_dtypes(include=['datetime64[ns]']).columns
    for col in cols:
        df[col] = df[col].dt.tz_localize(tz)
    return df


        


In [9]:
#Leer Excel
archivo = 'store_data.xlsx'
df = pd.read_excel(archivo, sheet_name='Orders')
df = df.reset_index(drop=True)

#Estandarizar nombre de columnas
df.columns = map(lambda x: str(x).upper(), df.columns)
df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
df

,ROW_ID,ORDER_PRIORITY,DISCOUNT,UNIT_PRICE,SHIPPING_COST,CUSTOMER_ID,CUSTOMER_NAME,SHIP_MODE,CUSTOMER_SEGMENT,PRODUCT_CATEGORY,...,REGION,STATE_OR_PROVINCE,CITY,POSTAL_CODE,ORDER_DATE,SHIP_DATE,PROFIT,QUANTITY_ORDERED_NEW,SALES,ORDER_ID
0,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2015-01-07,2015-01-08,4.5600,4,13.01,88522
1,20228,Not Specified,0.02,500.98,26.00,5,Ronnie Proctor,Delivery Truck,Home Office,Furniture,...,West,California,San Gabriel,91776,2015-06-13,2015-06-15,4390.3665,12,6362.85,90193
2,21776,Critical,0.06,9.48,7.29,11,Marcus Dunlap,Regular Air,Home Office,Furniture,...,East,New Jersey,Roselle,7203,2015-02-15,2015-02-17,-53.8096,22,211.15,90192
3,24844,Medium,0.09,78.69,19.99,14,Gwendolyn F Tyson,Regular Air,Small Business,Furniture,...,Central,Minnesota,Prior Lake,55372,2015-05-12,2015-05-14,803.4705,16,1164.45,86838
4,24846,Medium,0.08,3.28,2.31,14,Gwendolyn F Tyson,Regular Air,Small Business,Office Supplies,...,Central,Minnesota,Prior Lake,55372,2015-05-12,2015-05-13,-24.0300,7,22.23,86838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947,19842,High,0.01,10.90,7.46,3397,Andrea Shaw,Regular Air,Small Business,Office Supplies,...,Central,Illinois,Danville,61832,2015-03-11,2015-03-12,-116.7600,18,207.31,87536
1948,19843,High,0.10,7.99,5.03,3397,Andrea Shaw,Regular Air,Small Business,Technology,...,Central,Illinois,Danville,61832,2015-03-11,2015-03-12,-160.9520,22,143.12,87536
1949,26208,Not Specified,0.08,11.97,5.81,3399,Marvin Reid,Regular Air,Small Business,Office Supplies,...,Central,Illinois,Des Plaines,60016,2015-03-29,2015-03-31,-41.8700,5,59.98,87534
1950,24911,Medium,0.10,9.38,4.93,3400,Florence Gold,Express Air,Small Business,Furniture,...,East,West Virginia,Fairmont,26554,2015-04-04,2015-04-04,-24.7104,15,135.78,87537


In [10]:
table = 'store_orders_data'

with engine.connect() as con:
    df.to_sql(name=table, con=con, index=False, if_exists='replace', method=pd_writer)
    print(table + ' cargada en snowflake')

store_orders_data cargada en snowflake
